### Rough Heston model 
The rough Heston model is given by
   
   $dS_t = S_t\sqrt{V_t}dW_t $

   $V_t = V_0 + \frac{\gamma}{\Gamma(H + \frac{1}{2})}\int_{0}^{t}(t-s)^{H - \frac{1}{2}}(\theta - V_s)ds
 + \frac{\gamma \nu}{\Gamma(H + \frac{1}{2})}\int_{0}^{t}(t-s)^{H-\frac{1}{2}}\sqrt{V_s}dB_s$
 
   where $\gamma, \theta, \nu, \rho$ is the set of parameters, $W$ and $B$ are correlated with $\rho$, and $V_0$ is known. 
    
   Decompose the time interval $I = $\[$0,T$\] into $M$ uniform subintervals $I_n \colon= [t_n, t_{n+1}]$ for $n = 0, 1,\cdots,M-1$ of size $\tau = \frac{T}{M}$, $t_n = n \times \tau$, for $n=0,1,\cdots,M$.
   
   Note that $t^{H-\frac{1}{2}}\approx \sum_{j=1}^{N}\omega_je^{-\lambda_jt}$, where $\{\omega_j\}_{j=1}^{N}$ and $\{\lambda_j\}_{j=1}^{N}$ are unique for all $t \in (0, T]$
   
   For $V_{t_i}$, let $F_{a,i} \colon= \int_{0}^{t_i}(t_i-s)^{H-\frac{1}{2}}(\theta - V_s)ds$, $F_{b,i} \colon= \int_{0}^{t_i}(t_i-s)^{H - \frac{1}{2}}\sqrt{V_s}dB_s$.    

In [2]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt 
import time 
from scipy.optimize import bisect
from scipy.special import gamma 
from scipy import integrate

In [2]:
#sum of exponential approximation
#we get N, Omega, Lambda after this procedure


In [7]:
#BS formula 
def BS(flag, S0, K, T, r, vol):
    #for call option, flag = True; for put option, flag = False 
    #D: discount factor 
    #r: risk-free interest rate
    #F: forward price := S0/D

    vsqrt = vol*np.sqrt(T)
    D = np.exp(-r*T)
    F = S0/D
    d1 = np.log(F/K)/vsqrt + vsqrt/2
    d2 = d1 - vsqrt
    
    if Flag:
        return D*(F*norm.cdf(d1) - K*norm.cdf(d2))
    else:
        return D*(K*norm.cdf(-d2) - F*norm.cdf(-d1))
    
def impliedvol(C, S0, K, T, r): #by bisection algorithm
    #C: call option price 
    def smileMin(vol, *args):
        C, S0, K, T, r = args 
        return C - BS(True, S0, K, T, r, vol)
    vMin = 0.000001
    vMax = 10.
    return bisect(smileMin, vMin, vMax, args = (C, S0, K, T, r), 
                 rtol = 1e-15, full_output=False, dsip=True)



In [6]:
class rough_Heston_Analytical: 
    def __init__(self, M, T, params, Omega, Lambda, N, P):
        #Time discretization
        self.M = M #number of time intervals 
        self.T = T
        self.tau = self.T/self.M
        self.grid = np.linspace(0,T, self.M+1)
        self.Omega = Omega #(N,) 1-d array: 𝜔_1, 𝜔_2,...,𝜔_N
        self.Lambda = Lambda # (N,) 1-d array: 𝜆_1, 𝜆_2,...,𝜆_N       
        self.N = N
        self.P = P #number of paths to generate       
        
        #Heston model parameters 
        self.S0 = params['S0']
        self.V0 = params['V0']
        self.gamma = params['gamma']
        self.nu = params['nu']
        self.theta = params['theta']
        self.rho = params['rho']
        self.H = params['H']  #alpha = H + 1/2
        
        #Precomputation
        self.Omega_t = self.Omega/self.tau
        self.E = np.exp(-1*self.Lambda*self.tau)  
        self.coef_1 = self.gamma/gamma(self.H + 0.5)
        self.coef_2 = self.coef_1*self.nu
        self.C()
        self.Pi()
        
        
    #Brownian motions W and B
    def BW(self): 
        np.random.seed(0)
        self.W = np.sqrt(self.tau)*np.random.normal(loc=0, scale=1, size = [P,M]) 
        Z = np.sqrt(self.tau)*np.random.normal(loc=0, scale=1, size = [P,M]) 
        self.B = self.rho*self.W + np.sqrt(1-self.rho**2)*Z #W and B correlated with rho
        
    #compute C_1 and C_2 
    def C(self):
        self.C_1 = np.zeros((self.M, self.N))
        self.C_2 = np.zeros((self.M, self.N))
            
        #1st row of C_1 and C_2
        t1 = self.grid[1]
        for j in range(N):
            self.C_1[0,j] = integrate.quad(lambda x: np.exp(-1*self.Lambda[j]*(t1-x))*(t1-x), 0, t1)
            self.C_2[0,j] = integrate.quad(lambda x: np.exp(-1*self.Lambda[j]*(t1-x))*x, 0, t1)
        
        #2nd ∼ Mth rows of C_1 and C_2
        for i in range(1,M):
            t_a = self.grid[i]
            t_b = self.grid[i+1]
            for k in range(N):
                self.C_1[i,k] = integrate.quad(lambda x: np.exp(-1*self.Lambda[k](t_a-x))*(t_b-x), t_a, t_b)
                self.C_2[i,k] = integrate.quad(lambda x: np.exp(-1*self.Lambda[k](t_a-x))*(x-t_a), t_a, t_b)
        
    #compute Pi_1 and Pi_2
    def Pi(self): 
        self.Pi_1 = np.zeros((self.P, self.M, self.N)) #a 3-d matrix, or total P numbers of M×N 2-d matrices 
        self.Pi_2 = np.zeros((self.P, self.M, self.N)) 
        
        #1st row 
        t1 = self.grid[1]
        for i in range(P):
            for k in range(N):
                f_1 = integrate.quad(lambda x: np.exp(-2*self.Lambda[k]*(t1-x))*(t1-x)**2, 0, t1)
                f_2 = integrate.quad(lambda x: np.exp(-2*self.Lambda[k]*(t1-x))*x**2, 0, t1)
                self.Pi_1[i,0,k] = np.sqrt(f_1)*self.B[i, 0] 
                self.Pi_2[i,0,k] = np.sqrt(f_2)*self.B[i, 0]
            
        #2nd ∼ Mth rows 
        for i in range(P):
            for j in range(1,M):
                t_a = self.grid[j]
                t_b = self.grid[j+1]
                for k in range(N):
                    f_1 = integrate.quad(lambda x: np.exp(-2*self.Lambda[k]*(t_a-x))*(t_b-x)**2, t_a, t_b)
                    f_2 = integrate.quad(lambda x: np.exp(-2*self.Lambda[k]*(t_a-x))*(x-t_a)**2, t_a, t_b)
                    self.Pi_1[i,j,k] = np.sqrt(f_1)*self.B[i,j]
                    self.Pi_2[i,j,k] = np.sqrt(f_2)*self.B[i,j]                   
     
        
        
    def V(self):
        self.V = np.zeros((self.P, self.M))          
        G = np.zeros((self.P, self.M, self.N))
        Phi = np.zeros((self.P, self.M, self.N))
        
        #multipy Omega_t to C and Pi by row
        C_1_t = self.Omega_t*self.C_1
        C_2_t = self.Omega_t*self.C_2
        Pi_1_t = self.Omega_t*self.Pi_1
        Pi_2_t = self.Omega_t*self.Pi_2     

        #Compute 1st col of V
        m_1 = self.theta - self.V0
        C_s = m_1*C_1_t[0,:]
        C_a = m_1*C_2_t[0,:]
        
        m_2 = np.sqrt(self.V0)
        Pi_s_1 = m_2*Pi_1_t[:,0,:]
        Pi_a_1 = m_2*Pi_2_t[:,0,:]
        
        G[:,0,:] = C_s
        Phi[:,0,:] = Pi_s
        
        F_a = np.sum(G[:,0,:] + C_a)
        F_b = np.sum(Phi[:,0,:] + Pi_a, axis = -1)
        self.V[:,0] = self.V0 + self.coef_1*F_a + self.coef_2*F_b
        
        #Compute 2nd col of V
        m_1 = np.reshape(self.theta - self.V[:,0], (self.P, -1))  #P×1 array
        C_s = m_1 * np.reshape(C_1_t[1,:], (1,-1)) #reshape into a 1×N array
        C_a = m_1 * np.reshape(C_2_t[1,:], (1,-1))
        C_b = m_1 * np.reshape(C_2_t[0,:], (1,-1))
        
        m_2 = np.reshape(np.sqrt(self.V[:,0]), (self.P, -1)) 
        Pi_s = m_2 * Pi_1_t[:,1,:]
        Pi_a = m_2 * Pi_2_t[:,1,:]
        Pi_b = m_2 * Pi_2_t[:,0,:]
        
        G[:,1,:] = self.E*(G[:,0,:] + C_b + C_s)
        Phi[:,1,:] = self.E*(Phi[:,0,:] + Pi_b + Pi_s)
        
        F_a = np.sum(G[:,1,:] + self.E*C_a, axis = -1)
        F_b = np.sum(Phi[:,1,:] + self.E*Pi_a, axis = -1)
        self.V[:,1] = self.V0 + self.coef_1*F_a + self.coef_2*F_b
        
        #compute 3rd ∼ Mth cols of V
        for j in range(2, M):
            m_1 = np.reshape(self.theta - self.V[:,j-1], (self.P, -1))
            C_s = m_1 * np.reshape(C_1_t[j,:], (1,-1))
            C_a = m_1 * np.reshape(C_2_t[j,:], (1,-1))
            C_b = m_1 * np.reshape(C_2_t[j-1,:], (1,-1))
            
            m_2 = np.reshape(np.sqrt(self.V[:,j-1]), (self.P, -1))
            Pi_s = m_2 * Pi_1_t[:,j,:]
            Pi_a = m_2 * Pi_2_t[:,j,:]
            Pi_b = m_2 * Pi_2_t[:,j-1,:]
            
            G[:,j,:] = self.E*(G[:,j-1,:] + self.E*C_b + C_s)
            Phi[:,j,:] = self.E*(Phi[:,j-1,:] + self.E*Pi_b + Pi_s)
            
            F_a = np.sum(G[:,j,:] + self.E*C_a, axis = -1)
            F_b = np.sum(Phi[:,j,:] + self.E*Pi_a, axis = -1)
            self.V[:,j] = self.V0 + self.coef_1*F_a + self.coef_2*F_b
            
    def S(self): #by Forward Euler method 
        self.S = np.zeros((self.P, self.M)) 
        self.S[:,0] = self.S0 + self.S0*np.sqrt(self.V0)*self.W[:,0]
        
        for j in range(1, M):
            self.S[:,j] = self.S[:,j-1] + self.S[:,j-1]*np.sqrt(self.V[:,j-1])*self.W[:,j]
        
        return self.S
    
    def call_price_vol(self, k): 
        #k: log-moneyness 
        K = self.S0*np.exp(k) #K: strike 
        
        #pricing under BS formula 
        C = np.mean(BS(True, self.S[:,-1], self.T, 0, np.sqrt(self.V[:,-1])))       
        
        #implied_vol
        iv = impliedvol(C, self.S0, K, self.T, 0)
        
        return C, iv        
  